In [1]:
import numpy as np

import os
from IPython.display import Image
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import copy

from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import lifelines

import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
from keras import layers, initializers, optimizers, losses

from tensorflow.keras.callbacks import Callback
from tqdm.keras import TqdmCallback
from tqdm import tqdm

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config = config)

2025-10-16 15:39:48.246095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760639988.263783  109426 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760639988.269161  109426 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760639988.283900  109426 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760639988.283925  109426 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760639988.283928  109426 computation_placer.cc:177] computation placer alr

In [2]:
import weibull
import frailty_model_nn as fnn

Now that we have implemented a structure for training frailty survival models with neural networks components easily and with some basic functions at our disposal, let us define the discrete frailty model where
\begin{equation}
    Z \sim \text{Poisson}(W \theta); \ \ \ \ W \sim \text{Inverse-Gaussian}()
\end{equation}

In [ ]:
pvf_parameters = {
    "theta": {"link": lambda x : tf.math.exp(x), "link_inv": lambda x : tf.math.log(x), "par_type": "nn", "update_func": None, "shape": None, "initializer": None},
    # "tau": {"link": lambda x : tf.math.exp(x), "link_inv": lambda x : tf.math.log(x), "par_type": "nn", "update_func": None, "shape": 5, "initializer": None},
    "alpha": {"link": lambda x : 1/(1+tf.math.exp(-x)), "link_inv": lambda x : tf.math.log(x) - tf.math.log(1-x), "par_type": "independent", "update_func": None, "shape": None, "init": 0.5},
    "gamma": {"link": lambda x : tf.math.exp(x), "link_inv": lambda x : tf.math.log(x), "par_type": "independent", "update_func": None, "shape": None, "init": 1.0},
    "phi1": {"link": lambda x : tf.math.exp(x), "link_inv": lambda x : tf.math.log(x), "par_type": "independent", "update_func": None, "shape": None, "init": 1.0},
    "phi2": {"link": lambda x : tf.identity(x), "link_inv": lambda x : tf.identity(x), "par_type": "independent", "update_func": None, "shape": None, "init": 0.0}
}

def pvf_loglikelihood_loss(model, nn_output, t, delta):
        """
            This is an example of a correctly defined loss function.
        """
        theta = model.get_variable("theta", nn_output)

        alpha = model.get_variable("alpha")
        gamma = model.get_variable("gamma")
        phi1 = model.get_variable("phi1")
        phi2 = model.get_variable("phi2")

        log_f0 = tf.math.log(phi1) + (phi1-1)*tf.math.log(t) + phi2 - tf.math.exp(phi2) * tf.math.pow(t, phi1)
        F0 = 1 - tf.math.exp(-tf.math.pow(t, phi1) * tf.math.exp(phi2))

        laplace_transform_term = 1 + gamma*theta*F0/(1-alpha)

        loss_weights = delta*tf.math.log(theta) + delta*log_f0 + (1-alpha)/(alpha*gamma)*(1 - tf.math.pow(laplace_transform_term, alpha)) + (alpha-1)*delta*tf.math.log(laplace_transform_term)

        # It is very important to specify axis = [0,1] in this case, since the output shape for this function might be an array if we pass multiple parameters to it
        # That happens, for example, in the method plot_loglikelihood, that builds pairwise 3d surface plots for the log-likelihood automatically
        # The mean should always be taken in axis 0 and 1. The first relates to the data indices directly, while the second relates to the output dimension
        # of the neural network layers. The second dimension should always be equal to one. That means that even if we have vector parameter outputs from
        # the neural network component, the loglikelihood loss must return a single numeric value with shape () during training
        loss_weights_mean = -tf.math.reduce_mean(loss_weights, axis = [0,1])
        
        return loss_weights_mean

def pvf_neural_network(model, seed = None):
    """ 
        This is an example of a correctly defined neural network structure definition function.
    """
    initializer = initializers.GlorotNormal(seed = seed)
    model.dense1 = keras.layers.Dense(units = 16, activation = "gelu", kernel_initializer = initializer, dtype = tf.float32, name = "dense1")
    model.dense2 = keras.layers.Dense(units = 1, kernel_initializer = initializer, dtype = tf.float32, activation = None, use_bias = False, name = "output")

def pvf_neural_network_call(model, x_input):
    """
        This is an example of a correctly defined neural netowork call function. Its elements such as dense1 and dense2 must have been defined in neural_network_structure.
        Otherwise, it will result in an error
    """
    x = model.dense1(x_input)
    x = model.dense2(x)
    return x

In [ ]:
fnn.FrailtyModelNN()